## Pipeline example

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

In [4]:
from quantopian.pipeline.filters import Q1500US #top N most traded stocks (updated monthly, only primary shares)

In [5]:
universe = Q1500US()

In [6]:
from quantopian.pipeline.data import morningstar

In [27]:
# Classifiers = sort equities by category at a given time
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [9]:
energy_sector = sector.eq(309)

In [11]:
energy_sector.alias

<bound method ComputableTerm.alias of NumExprFilter(expr='x_0 == 309', bindings={'x_0': Latest([asset_classification<US>.morningstar_sector_code], 1)})>

In [12]:
# Masking - filters applied before actual calculation
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [13]:
dollar_volume = AverageDollarVolume(window_length = 30)

In [20]:
dollar_volume.top(5)

NumExprFilter(expr='x_0 <= (5.0)', bindings={'x_0': Rank(AverageDollarVolume(...), method='ordinal')})

In [21]:
high_dollar_volume = dollar_volume.percentile_between(90, 100)

In [22]:
high_dollar_volume

PercentileFilter([AverageDollarVolume(...)], 0)

In [24]:
top_open_prices = USEquityPricing.open.latest.top(50, mask = high_dollar_volume)

### Putting it all together in a single function

In [28]:
def make_pipeline():
    
    # Universe Q1500US
    base_universe = Q1500US()
    
    # Classifier: Energy Sector
    energy_sector = sector.eq(309)
    
    # Masking: 1500Us, Energy
    base_energy = base_universe & energy_sector
    
    # Dollar Volume info (30 days MA)
    dollar_volume = AverageDollarVolume(window_length = 30)
    
    # Grab top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    # Combine the filters
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 days mean close
    mean_10 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 10, mask = top_five_base_energy)
    
    # 30 days mean close
    mean_30 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 30, mask = top_five_base_energy)
    
    
    # Percent difference
    percent_difference = (mean_10 - mean_30) / mean_30
    
    # List of Shorts
    shorts = percent_difference < 0
    
    # List of Longs
    longs = percent_difference > 0
    
    # Final mask/filter for anything in shorts/longs
    securities_to_trade = (shorts | longs)
    
    # Return Pipeline
    return Pipeline (columns = {
        'longs': longs,
        'shorts': shorts,
        'perc_diff': percent_difference
    }, screen = securities_to_trade)

In [29]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

longs  perc_diff shorts
2015-05-05 00:00:00+00:00 Equity(216 [HES])     True   0.036528  False
                          Equity(448 [APA])     True   0.035926  False
                          Equity(455 [APC])     True   0.049153  False
                          Equity(858 [BHI])     True   0.033807  False
                          Equity(1746 [COG])    True   0.058078  False
                          Equity(2368 [DVN])    True   0.046264  False
                          Equity(2564 [EOG])    True   0.032102  False
                          Equity(2621 [VAL])    True   0.060197  False
                          Equity(3443 [HAL])    True   0.049257  False
                          Equity(3647 [HP])     True   0.040991  False
                          Equity(5035 [MRO])    True   0.061598  False
                          Equity(5213 [NBL])    True   0.010443  False
                          Equity(5214 [NBR])    True   0.064133  False
                          Equity(5249 [NE])     True   0.037559  False
                          Equity(5729 [OXY])    True   0.029776  False
                          Equity(6928 [SLB])    True   0.046555  False
                          Equity(7244 [SWN])    True   0.070788  False
                          Equity(7612 [ANDV])   True   0.005997  False
                          Equity(7990 [VLO])   False  -0.017145   True
                          Equity(8214 [WMB])    True   0.018876  False
                          Equity(8347 [XOM])    True   0.017343  False
                          Equity(8461 [CHK])    True   0.014265  False
                          Equity(9038 [RIG])    True   0.048180  False
                          Equity(13176 [CAM])   True   0.082110  False
                          Equity(17436 [PXD])   True   0.010248  False
                          Equity(19249 [RRC])   True   0.087062  False
                          Equity(19336 [WFT])   True   0.049141  False
                          Equity(22784 [FTI])   True   0.054529  False
                          Equity(23112 [CVX])   True   0.018972  False
                          Equity(23998 [COP])   True   0.023902  False
                          Equity(24809 [NOV])   True   0.024940  False
                          Equity(25707 [WLL])   True   0.048205  False
                          Equity(33856 [CLR])   True   0.064304  False
                          Equity(34440 [CXO])   True   0.042184  False
                          Equity(39797 [OAS])   True   0.042388  False
                          Equity(40852 [KMI])   True   0.023016  False
                          Equity(41636 [MPC])   True   0.011952  False
                          Equity(42788 [PSX])   True   0.020911  False

In [30]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 38 entries, (2015-05-05 00:00:00+00:00, Equity(216 [HES])) to (2015-05-05 00:00:00+00:00, Equity(42788 [PSX]))
Data columns (total 3 columns):
longs        38 non-null bool
perc_diff    38 non-null float64
shorts       38 non-null bool
dtypes: bool(2), float64(1)
memory usage: 684.0+ bytes
